In [4]:
import pandas as pd
import sqlite3

In [5]:
def run_query(query):
    return pd.read_sql_query(query,db)

In [6]:
db = sqlite3.connect('database.sqlite')

In [7]:
 run_query("SELECT tbl_name FROM sqlite_master;")

,tbl_name
0,station
1,status
2,trip
3,weather


In [6]:
run_query("SELECT * FROM trip LIMIT 5;")

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597


In [7]:
query = '''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE start_station_name = 'Paseo de San Antonio' 
LIMIT 5;
'''
run_query(query)

,start_station_name,end_station_name,duration
0,Paseo de San Antonio,Adobe on Almaden,12280
1,Paseo de San Antonio,San Jose Diridon Caltrain Station,454
2,Paseo de San Antonio,Paseo de San Antonio,1481
3,Paseo de San Antonio,Paseo de San Antonio,1475
4,Paseo de San Antonio,San Jose Diridon Caltrain Station,573


In [10]:
query = 'select max(duration) from trip;'
run_query(query)

,max(duration)
0,17270400


In [11]:
query = 'select count(*) from trip where end_station_name = "2nd at South Park";'
run_query(query)

,count(*)
0,16843


In [12]:
query = 'select count(*) from trip where duration > 1000 and duration < 2000'
run_query(query)

,count(*)
0,56705


In [13]:
query = 'select bike_id, count(bike_id) as count from trip group by bike_id order by count desc limit 5;'
run_query(query)

,bike_id,count
0,392,2061
1,489,1975
2,558,1955
3,267,1951
4,631,1948


In [14]:
query = 'select start_station_name, count(start_station_name) as count from trip where start_station_name = end_station_name group by start_station_name order by count desc limit 5;'
run_query(query)

,start_station_name,count
0,Embarcadero at Sansome,2092
1,Harry Bridges Plaza (Ferry Building),1733
2,University and Emerson,894
3,Market at 4th,724
4,2nd at Townsend,683


In [20]:
db = sqlite3.connect("my_database.db")

In [26]:
cursor = db.cursor()
cursor.execute(query)

In [27]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [28]:
results = cursor.fetchall()
print(results)

[('customer',)]


In [29]:
query = "INSERT INTO customer VALUES  (701, 'Mackenzie', 'Fox', 35)"
cursor.execute(query)

In [30]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35)]


In [31]:
cursor.execute("ALTER TABLE customer ADD COLUMN city TEXT DEFAULT 'Geneva';")
db.commit()

In [32]:
df2 = pd.DataFrame([[702, 'Emily', 'Joy', 49, 'Geneva']], columns=['id','first_name','last_name', 'age', 'city'])

In [33]:
df2.to_sql(name='customer', con=db, if_exists='append', index=False)

In [34]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35, 'Geneva'), (702, 'Emily', 'Joy', 49, 'Geneva')]


In [35]:
update = '''
UPDATE customer
SET city='Zurich'
WHERE id=702;
'''

In [37]:
cursor.execute(update)
db.commit()

In [38]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35, 'Geneva'), (702, 'Emily', 'Joy', 49, 'Zurich')]


In [39]:
db = sqlite3.connect('songs.db')
cursor= db.cursor()

In [40]:
for chunk in pd.read_csv("songs.csv", chunksize=4):
    chunk.to_sql(name="data", con=db, if_exists="append", index=False)  
    print(chunk.iloc[0, 2])

Stairway to Heaven
Black Dog
All My Love
Rebel Rebel
Golden Years


In [41]:
pd.read_sql_query('SELECT * FROM data;', db)

,Musician,Genre,Name,Decade,Minutes
0,Led Zeppelin,hard rock,Stairway to Heaven,70,08:02
1,Led Zeppelin,hard rock,Kashmir,70,08:37
2,Led Zeppelin,hard rock,Immigrant Song,70,02:26
3,Led Zeppelin,hard rock,Whole Lotta Love,60,05:33
4,Led Zeppelin,hard rock,Black Dog,70,04:55
5,Led Zeppelin,hard rock,Good Times Bad Times,60,02:43
6,Led Zeppelin,hard rock,Moby Dick,60,04:25
7,Led Zeppelin,hard rock,Ramble On,60,04:35
8,Led Zeppelin,hard rock,All My Love,70,05:53
9,Led Zeppelin,hard rock,The Song Remains the Same,70,05:24


In [42]:
complaints = pd.read_csv("consumer_complaints.csv")

In [43]:
complaints.to_sql(name="complaints",con=db,index=False)

In [44]:
pd.read_sql_query('SELECT * FROM complaints;', db)

,complaint_id,product,issue,company,state,zipcode,submitted_via,date_sent_to_company,date_received,company_response_to_consumer,timely_response,consumer_disputed?
0,511074,Mortgage,"Loan modification,collection,foreclosure",U.S. Bancorp,CA,95993,Referral,09/03/2013,08/30/2013,Closed with explanation,Yes,Yes
1,511080,Mortgage,"Loan servicing, payments, escrow account",Wells Fargo & Company,CA,91104,Referral,09/03/2013,08/30/2013,Closed with explanation,Yes,Yes
2,510473,Credit reporting,Incorrect information on credit report,Wells Fargo & Company,NY,11764,Postal mail,09/18/2013,08/30/2013,Closed with explanation,Yes,No
3,510326,Student loan,Repaying your loan,"Navient Solutions, Inc.",MD,21402,Email,08/30/2013,08/30/2013,Closed with explanation,Yes,Yes
4,511067,Debt collection,False statements or representation,Resurgent Capital Services L.P.,GA,30106,Web,08/30/2013,08/30/2013,Closed with explanation,Yes,Yes
5,510098,Credit card,Application processing delay,Capital One,NY,12206,Phone,09/03/2013,08/30/2013,Closed with explanation,Yes,Yes
6,511062,Credit card,Credit line increase/decrease,Wells Fargo & Company,AZ,85730,Postal mail,09/05/2013,08/30/2013,Closed with explanation,Yes,No
7,511116,Bank account or service,Deposits and withdrawals,Bank of America,IL,60660,Referral,09/04/2013,08/30/2013,Closed with explanation,Yes,No
8,511091,Bank account or service,Deposits and withdrawals,Bank of America,GA,30016,Referral,09/04/2013,08/30/2013,Closed with explanation,Yes,No
9,530602,Mortgage,"Loan modification,collection,foreclosure","SunTrust Banks, Inc.",CA,94551,Web,09/18/2013,09/17/2013,Closed with explanation,Yes,Yes


In [45]:
pd.read_sql_query('SELECT count(*) FROM complaints;', db)

,count(*)
0,65499


In [54]:
query = 'Select submitted_via as Method, count(*) as Number, (SELECT count(*) from complaints) as Total from complaints group by submitted_via;'
summary = pd.read_sql_query(query, db)

In [57]:
summary["Percent"] = summary["Number"] / summary["Total"] * 100

In [58]:
summary

,Method,Number,Total,Percent
0,Email,21,65499,0.032062
1,Fax,1076,65499,1.642773
2,Phone,4974,65499,7.594009
3,Postal mail,4894,65499,7.471870
4,Referral,10587,65499,16.163606
5,Web,43947,65499,67.095681


In [52]:
summary

,Method,Number,Total
0,Email,21,65499
1,Fax,1076,65499
2,Phone,4974,65499
3,Postal mail,4894,65499
4,Referral,10587,65499
5,Web,43947,65499


In [50]:
summary

,Method,Number
0,Email,21
1,Fax,1076
2,Phone,4974
3,Postal mail,4894
4,Referral,10587
5,Web,43947


In [24]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

AttributeError: 'builtin_function_or_method' object has no attribute 'execute'